In [83]:
import torch
import gc
import numpy as np
from IPython.display import clear_output
import torch.nn.functional as F
from IPython.display import display, Markdown, Latex
import time
from tqdm import tqdm
from accelerate import init_empty_weights
import matplotlib.pyplot as plt

In [84]:
gc.enable()

In [85]:
#!pip install accelerate

In [86]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# device = 'cuda'
device = 'cpu'
#model_name = 'EleutherAI/gpt-neo-1.3B'
model_name = 'EleutherAI/pythia-410m'
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float32)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [87]:
model.to(device)
model.eval()
pass

In [88]:
@torch.no_grad()
def generate_response_greedy(input_text, pre_prompt, break_word,max_length=300,temp=0.1, name='',
                            past_key_vals = None, next_id=None):

#     print(pre_prompt, input_text)
    if past_key_vals is None:
        inputs = tokenizer.encode(pre_prompt + input_text + '\n' + name, return_tensors="pt")
        response_ids = inputs
        length_prompt = len(response_ids[0])
        output = ''
        last_n = ''
    else:
        inputs = tokenizer.encode(input_text + '\n' + name, return_tensors="pt")
        response_ids = torch.concat((next_id, inputs),dim=-1)
        length_prompt = len(response_ids[0])
        output = ''
        last_n = ''
    print(name, end='')
    all_out = name
    for _ in (range(max_length)):
        out = model.forward(input_ids=response_ids.to(device), past_key_values=past_key_vals)
#         next_token_id = out.logits[:, -1, :].argmax(-1,keepdim=True)
        next_token_id = torch.multinomial(F.softmax(out.logits[:, -1, :]/temp,  dim=-1), num_samples=1).to('cpu')
        past_key_vals = out.past_key_values
        response_ids = next_token_id
        output = tokenizer.decode([response_ids[0][-1]], skip_special_tokens=True)
        all_out += output
#         clear_output(wait=True)
        print(output, end='')
        
        if output == break_word:
            break
        
#         display(Markdown(all_out))
        
        last_n += output
        last_n = last_n[-len(break_word):]
        if last_n == break_word:
            break
    decoded_output = tokenizer.decode(response_ids[0], skip_special_tokens=True)
    past_kv = past_key_vals
    next_id = response_ids
    return decoded_output.replace(pre_prompt, '').replace(input_text, ''), past_kv, next_id


In [34]:
#importing customer service prompt

with open(r'C:\Users\A1D\Desktop\NLP\NLP ASSIGNMENT 4\Customer Service\CUSTOMER_SERVICE_PREPROMT.txt') as f:
    pre_prompt = f.read()

In [50]:
pre_prompt = '''[USER]: How do I place an order on your website?
[BOT]: Shopping is super easy now. Register your Account today and start shopping by following these steps. 1. select item. 2. click buy now/ add to cart. 3. proceed to payment.4. select payment method[END]
[USER]: What are the payment options available?
[BOT]: Goods can be ordered via pre-payment methods and via COD. Below are the payment methods available for Digital Goods:Credit/Debit Card ,Bank Direct Transfer and e-Wallet[END]
[USER]: How can I track my order?
[BOT]: You can track your order from app. If you have trackind id you can track your order by entering trackid id on Webpage of respective courier service.[END]
[USER]: Can I cancel or modify my order?
[BOT]: Yes! you can cancel or modify your order.[END]
[USER]: What is your return or exchange policy?
[BOT]: The amount is refunded in the following scenarios: Refund against Return orders: the refund is processed if your return claim is deemed valid. Refund against Cancelled orders: Refund is automatically triggered once the cancellation has been successfully processed.[END]
[USER]: How long does it take for my order to be delivered?
[BOT]: You will receive your digital goods on the registered email address within 24hrs. Physical items mostly take 2-3 days for delivery.[END]
[USER]: What should I do if my order is damaged or defective?
[BOT]: Provide us with your order number and any relevant details, such as photos or videos of the issue, so that we can investigate and determine the best course of action. We may offer a refund, replacement item, or repair service depending on the circumstances.[END]
[USER]: Can I get a refund if I'm not satisfied with my purchase? 
[BOT]: Yes! you can ger refund. We do offer refunds to customers who are not satisfied with their purchase.We want our customers to be completely happy with their orders and will work with them to make thingsright.[END]
[USER]: How can I contact your customer service team?
[BOT]: You can call us at XXX-XXX-XXXXXX anytime ( Mon - Sun ) or to connect to our Seller Support teamon Chat between 10:00 AM - 4:00 PM ( Mon - Sun). You can also mail us at support.abc@xyz.com.[END]
[USER]: Do you offer any discounts or promotions?
[BOT]: You can avail discounts by using vouchers. You can collect vouchers by making purchases andperforming tasks.[END]
[USER]: Is my personal and payment information secure on your website?
[BOT]: we take the security of our customers' personal and payment information very seriously. We use a variety of security measures to protect our website and the information you provide to us.We use SSL encryption to protect your personal and payment information during transmission, and we do not store your payment information on our servers. We use a third-party payment processor that is PCI-compliant to handle all payment transactions, which means your payment information is processed securely and in compliance with industry standards.[END]
[USER]: What are the shipping costs?
[BOT]: Shipping costs vary depending on factors such as the weight and size of the item, the shipping method selected, and the destination.  I suggest checking the product page or contacting customer support for more information on the shipping costs for your order.[END]
[USER]: How do I know if my order has been shipped?
[BOT]: you can typically check the status of your order by logging into your account on the website or by checking the confirmation email you received when you placed your order. The seller may also send you a tracking number once your order has been shipped, which you can use to track the status of your shipment online. If you have any questions or concerns about the status of your order, you can contact customer support for assistance.[END]
[USER]: Do you offer international shipping?
[BOT]: we may offer international shipping to certain countries. To find out if we offer international shipping to your location, you can check the shipping information on our website or during the checkout process. If your country is not listed as an available shipping destination, we may not be able to fulfill your order at this time. However, we are constantly working to expand our shipping capabilities and may be able to offer international shipping to your location in the future. Please feel free to contact our customer support team for more information about our shipping policies and options.[END]
[USER]: How do I leave feedback or a review about my purchase?
[BOT]: we value our customers' feedback and opinions. To leave feedback or a review about your purchase, you can navigate to the product page on our website and look for the "Feedback" section. From there, you can leave a rating, write a review, and share your experience with other customers. Your feedback helps us improve our products and services.[END]
[USER]: Can I place an order over the phone?
[BOT]: we do not offer phone orders at this time. However, you can easily place an order on our website using your computer or mobile device. Our website is designed to be user-friendly and easy to navigate, making it simple for you to find the products you need and complete your purchase.[END]
[USER]: How do I use a coupon or promo code?
[BOT]: During the checkout process, you will have the opportunity to enter your coupon code in a designated field. The field  is labeled as "Coupon Code". After you have entered the code, click "Apply" or "Redeem" to apply the discount to your order. The discounted price should then be reflected in your order total.If you still have any query, please contact customer support.[END]
[USER]: Do you have a loyalty program or rewards system?
[BOT]: You can earn points for every purchase you do. These points can be used for discounts in your future purchases.[END]
[USER]: Can I get assistance with product selection?
[BOT]: If you need assistance in choosing a product, our website have product descriptions, reviews, and other helpful information to guide your decision. Additionally, you can contact our customer support team for personalized assistance.[END]
[USER]: Do you offer gift wrapping or gift cards?
[BOT]: we may offer gift wrapping and/or gift cards for purchase. Gift wrapping and gift cards can make great additions to your purchase, especially if you are buying a gift for someone else. You can check our website or contact our customer support team to find out if we offer these services and how you can purchase them.[END]
[USER]: What happens if my package is lost or stolen?
[BOT]: we understand that a lost or stolen package can be frustrating and inconvenient. In the event that your package is lost or stolen, we recommend that you contact our customer support team as soon as possible to report the issue. We will work with the shipping carrier to track down your package and try to resolve the issue. If your package cannot be located, we may offer a refund or a replacement shipment, depending on the circumstances. In some cases, we may require additional information or documentation to process a refund or replacement shipment. We encourage our customers to purchase shipping insurance or request signature confirmation for their shipments to help prevent lost or stolen packages. Our priority is to ensure that you receive your order in a timely and secure manner, and we will do everything we can to help you in the event of a lost or stolen package.[END]
[USER]: How do I unsubscribe from marketing emails?
[BOT]: we understand that some customers may want to unsubscribe from marketing emails. To do so, you can typically find an "Unsubscribe" link at the bottom of our marketing emails. Clicking on the link will take you to a page where you can confirm your request to unsubscribe from future emails. Please note that it may take a few days for your request to be processed, and you may still receive transactional emails related to your orders even if you have unsubscribed marketing emails. If you have any issues unsubscribing from our marketing emails or continue to receive unwanted emails, please contact our customer support team for assistance. We respect our customers' preferences and privacy and are committed to maintaining compliance with applicable laws and regulations regarding email marketing.[END]
[USER]: Are there any restrictions on shipping to my location?
[BOT]: there may be restrictions on shipping to certain locations due to legal, logistical, or other reasons. We recommend that you check our website or contact our customer support team to find out if we are able to ship to your location and if any restrictions apply. Some common reasons for shipping restrictions may include customs regulations, import restrictions, and hazardous materials restrictions. If your location is restricted, we apologize for any inconvenience and encourage you to explore alternative shipping options or products that are available in your region. Our priority is to ensure that we comply with all applicable laws and regulations and provide a positive shopping experience for all of our customers.[END]
[USER]: How do I initiate a warranty claim for a product?
[BOT]: if you need to initiate a warranty claim for a product purchased on our website, we recommend that you contact the manufacturer or the brand directly, as they are typically responsible for handling warranty claims. You can usually find contact information for the manufacturer or the brand on the product packaging, the instruction manual, or their website. When contacting the manufacturer or the brand, be sure to provide them with your proof of purchase, such as your order confirmation or receipt, and a description of the issue you are experiencing with the product. The manufacturer or the brand may require additional information or documentation to process your warranty claim, and they may provide you with instructions on how to return the product for repair or replacement.[END]
[USER]: Can I get assistance with technical issues or problems with the website?
[BOT]: If you are experiencing technical issues or problems with our website, please contact our customer support team for assistance. We will troubleshoot the issue and provide a solution as quickly as possible.[END]
[USER]: What happens if an item is out of stock after I place an order?
[BOT]: If an item is out of stock after you place an order, we will notify you and provide options to replace, wait, or cancel the item. We aim to provide the best shopping experience but in rare cases, an item may become unexpectedly out of stock due to high demand.[END]
[USER]: Do you offer any additional services, such as installation or assembly?
[BOT]: Yes, we may offer additional services such as installation or assembly for selected products. Please check our website or contact our customer support team for more information about these services and their availability.[END]
[USER]: How do I access my account or order history?
[BOT]: our website have a dedicated section for order history within your account dashboard. To access your account order history, Click on the "Order History" tab to view your order history.[END]
[USER]: Can I request expedited shipping?
[BOT]: we offer expedited shipping options for selected products. During the checkout process, you are given the option to choose expedited shipping and the associated costs. Please note that expedited shipping may not be available for all products or locations, and additional fees may apply.[END]
[USER]: What is the process for ordering custom or personalized products?
[BOT]: you can begin by selecting the product you wish to customize and choosing the customization options, such as color, size, text, or graphics. You may also need to upload any images or designs you wish to use for the customization.Once you have selected the customization options and uploaded any necessary files, you can add the product to your cart and proceed to checkout. During the checkout process, you may be prompted to review and approve a digital proof of the customized product before it goes into production. Please note that the production and shipping times for custom or personalized products may be longer than for standard products due to the additional customization process.[END]
[USER]: How can I obtain a copy of my invoice or receipt?
[BOT]: you can obtain a copy of your invoice or receipt by logging into your account on our website and navigating to your order history. From there, you should be able to view and download a copy of your invoice or receipt.[END]
[USER]: Do you offer any financing options for large purchases?
[BOT]: We offers monthly installments for large purchases. We offer different installment plans. You can check them by clicking on "MONTHLY INSTALLMENTS" tab.[END]
[USER]: Can I get assistance with sizing or fit questions?
[BOT]: we understand that finding the right size and fit can be difficult when shopping online. That's why we offer assistance with sizing and fit questions to help our customers make informed purchasing decisions.[END]
[USER]: Are there any restrictions on returns or exchanges?
[BOT]: we do have some restrictions on returns or exchanges.Firstly, we require that items are returned in their original condition and packaging. This means that items must be unworn, unused, and free from any signs of damage or wear.Secondly, some items may not be eligible for returns or exchanges, such as personalized or custom-made items. We also may not accept returns on items that have been marked as final sale or clearance.Additionally, we may require proof of purchase, such as a receipt or order confirmation email, to process your return or exchange.[END]
[USER]: How do I know if a product is eligible for free shipping?
[BOT]: we usually display information about our shipping policies on the product page, checkout page or in our shipping policy page.You can check eligibility from there.[END]
[USER]: How do I know if a product is eligible for free shipping?
[BOT]: we usually display information about our shipping policies on the product page, checkout page or in our shipping policy page.You can check eligibility from there.[END]
[USER]: Do you offer any price matching or price adjustment policies?
[BOT]: Yes, we may offer price matching or price adjustment policies under certain circumstances.We value our customers and want them to be satisfied with their purchases, and we'll do our best to help them get the best value for their money.[END]
[USER]: Can I order by mail or fax?
[BOT]: Sorry! Order cannot be placed by mail or fax. We primarily operate as an online platform and encourage customers to place orders through our website. Customers can easily access your website from PC and mobile phones.[END]
[USER]: How do I leave a product review on your website?
[BOT]: To leave a product review on our website, go to the product page, scroll down to the "Reviews" section, click on "Write a Review", and fill out the review form with your feedback and rating.[END]
[USER]: Can I request a specific delivery date or time?
[BOT]: We may provide delivery date or time options during checkout, but availability depends on shipping carrier and options.So, it is difficult to provide an exact date and time.[END]
[USER]: How can I get my product serviced?
[BOT]: If your product is within the warranty duration and malfunctions during normal usage, you do not have to worry about the repairing cost. If your product has passed the warranty duration, or is physically damaged, or violates a warranty condition as set out by the manufacturer, service facility will not be provided.[END]
[USER]: What is the process for ordering a product that is not currently listed on your website?
[BOT]:if you're interested in ordering a product that is not currently listed on our website, please contact our customer service team for assistance. Our team will work with you to gather the necessary information about the product you're looking for, such as the product name, brand, and specifications. Once we have this information, we will search our supplier network to see if we can source the product for you. If we're able to source the product, we will provide you with a quote and estimated delivery time. If you agree to the quote and delivery time, we will send you an invoice and you can complete the order through our website.Please note that the availability and pricing of products that are not listed on our website may vary based on supplier availability and market conditions. Our customer service team will do their best to provide you with accurate information and help you complete your order.[END]
[USER]: Do you offer any educational resources or product guides?
[BOT]: we understand that customers may have questions about our products and may need assistance with making informed purchasing decisions. We offer a variety of educational resources and product guides to help our customers make the best choices.On our website, we provide detailed product descriptions and specifications, as well as customer reviews and ratings to help you understand the features and benefits of our products. [END]
[USER]: How can I access customer service outside of normal business hours?
[BOT]: we understand that our customers may need assistance outside of our normal business hours. To ensure that you can access customer service at any time, we offer several options for contacting us outside of normal business hours. These options  include:Email support: You can send an email to our customer service team at any time and we will respond to your inquiry as soon as possible.Phone support: We may offer a 24/7 phone support line that you can call for assistance.Self-service options: Our website offer self-service options, such as FAQs or online tutorials, that you can access at any time to get answers to common questions.[END]
[USER]: Are there any additional fees or taxes added to my purchase?
[BOT]: As a representative of our ecommerce website, we strive to be transparent about all fees and taxes associated with your purchase.In general, the price listed on our website for each product is the final price you will pay at checkout, including any applicable taxes and fees. However, there may be certain circumstances where additional fees may apply, such as for expedited shipping or special handling requests.If any additional fees or taxes are applicable to your purchase, they will be clearly listed on the checkout page before you complete your purchase. Additionally, you will receive an itemized receipt that includes all charges associated with your purchase.[END]
[USER]: What is the process for canceling a pre-order or backorder?
[BOT]: To cancel a pre-order or backorder, please contact our customer service team as soon as possible. They will review your order and provide you with options for cancellation, which may include receiving a full refund or incurring certain fees or restrictions.'''

In [52]:
short_pre_prompt = '''
[USER]: How do I place an order on your website?
[BOT]: Shopping is super easy now. Register your Account today and start shopping by following these steps
select item
click buy now/ add to cart
proceed to payment
select payment method
[USER]: What are the payment options available?[END]
[BOT]: Goods can be ordered via pre-payment methods and via COD. Below are the payment methods available for Digital Goods:Credit/Debit Card ,Bank Direct Transfer and e-Wallet
[USER]: How can I track my order?[END]
[BOT]: You can track your order from app. If you have trackind id you can track your order by entering trackid id on Webpage of respective courier service.
[USER]: Can I cancel or modify my order?[END]
[BOT]: Yes! you can cancel or modify your order.[END]
'''

In [89]:
new_pre_prompt='''
[USER]: How do I place an order on your website?
[BOT]: Shopping is super easy now. Register your Account today and start shopping by following these steps. 1. select item. 2. click buy now/ add to cart. 3. proceed to payment.4. select payment method[END]
[USER]: What are the payment options available?
[BOT]: Goods can be ordered via pre-payment methods and via COD. Below are the payment methods available for Digital Goods:Credit/Debit Card ,Bank Direct Transfer and e-Wallet[END]
[USER]: How can I track my order?
[BOT]: You can track your order from app. If you have trackind id you can track your order by entering trackid id on Webpage of respective courier service.[END]
[USER]: Can I cancel or modify my order?
[BOT]: Yes! you can cancel or modify your order.[END]
[USER]: What is your return or exchange policy?
[BOT]: The amount is refunded in the following scenarios: Refund against Return orders: the refund is processed if your return claim is deemed valid. Refund against Cancelled orders: Refund is automatically triggered once the cancellation has been successfully processed.[END]
[USER]: How long does it take for my order to be delivered?
[BOT]: You will receive your digital goods on the registered email address within 24hrs. Physical items mostly take 2-3 days for delivery.[END]
[USER]: What should I do if my order is damaged or defective?
[BOT]: Provide us with your order number and any relevant details, such as photos or videos of the issue, so that we can investigate and determine the best course of action. We may offer a refund, replacement item, or repair service depending on the circumstances.[END]
[USER]: Can I get a refund if I'm not satisfied with my purchase? 
[BOT]: Yes! you can ger refund. We do offer refunds to customers who are not satisfied with their purchase.We want our customers to be completely happy with their orders and will work with them to make thingsright.[END]
[USER]: How can I contact your customer service team?
[BOT]: You can call us at XXX-XXX-XXXXXX anytime ( Mon - Sun ) or to connect to our Seller Support teamon Chat between 10:00 AM - 4:00 PM ( Mon - Sun). You can also mail us at support.abc@xyz.com.[END]
[USER]: Do you offer any discounts or promotions?
[BOT]: You can avail discounts by using vouchers. You can collect vouchers by making purchases andperforming tasks.[END]
[USER]: Is my personal and payment information secure on your website?
[BOT]: we take the security of our customers' personal and payment information very seriously. We use a variety of security measures to protect our website and the information you provide to us.We use SSL encryption to protect your personal and payment information during transmission, and we do not store your payment information on our servers. We use a third-party payment processor that is PCI-compliant to handle all payment transactions, which means your payment information is processed securely and in compliance with industry standards.[END]
[USER]: What are the shipping costs?
[BOT]: Shipping costs vary depending on factors such as the weight and size of the item, the shipping method selected, and the destination.  I suggest checking the product page or contacting customer support for more information on the shipping costs for your order.[END]
[USER]: How do I know if my order has been shipped?
[BOT]: you can typically check the status of your order by logging into your account on the website or by checking the confirmation email you received when you placed your order. The seller may also send you a tracking number once your order has been shipped, which you can use to track the status of your shipment online. If you have any questions or concerns about the status of your order, you can contact customer support for assistance.[END]
[USER]: Do you offer international shipping?
[BOT]: we may offer international shipping to certain countries. To find out if we offer international shipping to your location, you can check the shipping information on our website or during the checkout process. If your country is not listed as an available shipping destination, we may not be able to fulfill your order at this time. However, we are constantly working to expand our shipping capabilities and may be able to offer international shipping to your location in the future. Please feel free to contact our customer support team for more information about our shipping policies and options.[END]
[USER]: How do I leave feedback or a review about my purchase?
[BOT]: we value our customers' feedback and opinions. To leave feedback or a review about your purchase, you can navigate to the product page on our website and look for the "Feedback" section. From there, you can leave a rating, write a review, and share your experience with other customers. Your feedback helps us improve our products and services.[END]
[USER]: Can I place an order over the phone?
[BOT]: we do not offer phone orders at this time. However, you can easily place an order on our website using your computer or mobile device. Our website is designed to be user-friendly and easy to navigate, making it simple for you to find the products you need and complete your purchase.[END]
[USER]: How do I use a coupon or promo code?
[BOT]: During the checkout process, you will have the opportunity to enter your coupon code in a designated field. The field  is labeled as "Coupon Code". After you have entered the code, click "Apply" or "Redeem" to apply the discount to your order. The discounted price should then be reflected in your order total.If you still have any query, please contact customer support.[END]
[USER]: Do you have a loyalty program or rewards system?
[BOT]: You can earn points for every purchase you do. These points can be used for discounts in your future purchases.[END]
[USER]: Can I get assistance with product selection?
[BOT]: If you need assistance in choosing a product, our website have product descriptions, reviews, and other helpful information to guide your decision. Additionally, you can contact our customer support team for personalized assistance.[END]
[USER]: Do you offer gift wrapping or gift cards?
[BOT]: we may offer gift wrapping and/or gift cards for purchase. Gift wrapping and gift cards can make great additions to your purchase, especially if you are buying a gift for someone else. You can check our website or contact our customer support team to find out if we offer these services and how you can purchase them.[END]
[USER]: What happens if my package is lost or stolen?
[BOT]: we understand that a lost or stolen package can be frustrating and inconvenient. In the event that your package is lost or stolen, we recommend that you contact our customer support team as soon as possible to report the issue. We will work with the shipping carrier to track down your package and try to resolve the issue. If your package cannot be located, we may offer a refund or a replacement shipment, depending on the circumstances. In some cases, we may require additional information or documentation to process a refund or replacement shipment. We encourage our customers to purchase shipping insurance or request signature confirmation for their shipments to help prevent lost or stolen packages. Our priority is to ensure that you receive your order in a timely and secure manner, and we will do everything we can to help you in the event of a lost or stolen package.[END]
[USER]: How do I unsubscribe from marketing emails?
[BOT]: we understand that some customers may want to unsubscribe from marketing emails. To do so, you can typically find an "Unsubscribe" link at the bottom of our marketing emails. Clicking on the link will take you to a page where you can confirm your request to unsubscribe from future emails. Please note that it may take a few days for your request to be processed, and you may still receive transactional emails related to your orders even if you have unsubscribed marketing emails. If you have any issues unsubscribing from our marketing emails or continue to receive unwanted emails, please contact our customer support team for assistance. We respect our customers' preferences and privacy and are committed to maintaining compliance with applicable laws and regulations regarding email marketing.[END]
[USER]: Are there any restrictions on shipping to my location?
[BOT]: there may be restrictions on shipping to certain locations due to legal, logistical, or other reasons. We recommend that you check our website or contact our customer support team to find out if we are able to ship to your location and if any restrictions apply. Some common reasons for shipping restrictions may include customs regulations, import restrictions, and hazardous materials restrictions. If your location is restricted, we apologize for any inconvenience and encourage you to explore alternative shipping options or products that are available in your region. Our priority is to ensure that we comply with all applicable laws and regulations and provide a positive shopping experience for all of our customers.[END]
[USER]: How do I initiate a warranty claim for a product?
[BOT]: if you need to initiate a warranty claim for a product purchased on our website, we recommend that you contact the manufacturer or the brand directly, as they are typically responsible for handling warranty claims. You can usually find contact information for the manufacturer or the brand on the product packaging, the instruction manual, or their website. When contacting the manufacturer or the brand, be sure to provide them with your proof of purchase, such as your order confirmation or receipt, and a description of the issue you are experiencing with the product. The manufacturer or the brand may require additional information or documentation to process your warranty claim, and they may provide you with instructions on how to return the product for repair or replacement.[END]
[USER]: Can I get assistance with technical issues or problems with the website?
[BOT]: If you are experiencing technical issues or problems with our website, please contact our customer support team for assistance. We will troubleshoot the issue and provide a solution as quickly as possible.[END]
[USER]: What happens if an item is out of stock after I place an order?
[BOT]: If an item is out of stock after you place an order, we will notify you and provide options to replace, wait, or cancel the item. We aim to provide the best shopping experience but in rare cases, an item may become unexpectedly out of stock due to high demand.[END]
[USER]: Do you offer any additional services, such as installation or assembly?
[BOT]: Yes, we may offer additional services such as installation or assembly for selected products. Please check our website or contact our customer support team for more information about these services and their availability.[END]
[USER]: How do I access my account or order history?
[BOT]: our website have a dedicated section for order history within your account dashboard. To access your account order history, Click on the "Order History" tab to view your order history.[END]
[USER]: Can I request expedited shipping?
[BOT]: we offer expedited shipping options for selected products. During the checkout process, you are given the option to choose expedited shipping and the associated costs. Please note that expedited shipping may not be available for all products or locations, and additional fees may apply.[END]
[USER]: What is the process for ordering custom or personalized products?
[BOT]: you can begin by selecting the product you wish to customize and choosing the customization options, such as color, size, text, or graphics. You may also need to upload any images or designs you wish to use for the customization.Once you have selected the customization options and uploaded any necessary files, you can add the product to your cart and proceed to checkout. During the checkout process, you may be prompted to review and approve a digital proof of the customized product before it goes into production. Please note that the production and shipping times for custom or personalized products may be longer than for standard products due to the additional customization process.[END]
[USER]: How can I obtain a copy of my invoice or receipt?
[BOT]: you can obtain a copy of your invoice or receipt by logging into your account on our website and navigating to your order history. From there, you should be able to view and download a copy of your invoice or receipt.[END]
[USER]: Do you offer any financing options for large purchases?
[BOT]: We offers monthly installments for large purchases. We offer different installment plans. You can check them by clicking on "MONTHLY INSTALLMENTS" tab.[END]
[USER]: Can I get assistance with sizing or fit questions?
[BOT]: we understand that finding the right size and fit can be difficult when shopping online. That's why we offer assistance with sizing and fit questions to help our customers make informed purchasing decisions.[END]
[USER]: Are there any restrictions on returns or exchanges?
[BOT]: we do have some restrictions on returns or exchanges.Firstly, we require that items are returned in their original condition and packaging. This means that items must be unworn, unused, and free from any signs of damage or wear.Secondly, some items may not be eligible for returns or exchanges, such as personalized or custom-made items. We also may not accept returns on items that have been marked as final sale or clearance.Additionally, we may require proof of purchase, such as a receipt or order confirmation email, to process your return or exchange.[END]
[USER]: How do I know if a product is eligible for free shipping?
[BOT]: we usually display information about our shipping policies on the product page, checkout page or in our shipping policy page.You can check eligibility from there.[END]
[USER]: How do I know if a product is eligible for free shipping?
[BOT]: we usually display information about our shipping policies on the product page, checkout page or in our shipping policy page.You can check eligibility from there.[END]
'''

# creating train data

In [37]:
train_data =[]
conversation=[]    
d={}
s=[]
i= 0
for line in pre_prompt.split('\n'):
    if i ==0:
        d['prompt'] = line
        s.append(line)
        i=1
    else:
        s.append(line)
        d['response'] = line
        train_data.append(d)
        conversation.append(s)
        s=[]
        d={}
        i=0

In [38]:
train_data

[{'prompt': '[USER]: How do I place an order on your website?',
  'response': '[BOT]: Shopping is super easy now. Register your Account today and start shopping by following these steps. 1. select item. 2. click buy now/ add to cart. 3. proceed to payment.4. select payment method'},
 {'prompt': '[USER]: What are the payment options available?',
  'response': '[BOT]: Goods can be ordered via pre-payment methods and via COD. Below are the payment methods available for Digital Goods:Credit/Debit Card ,Bank Direct Transfer and e-Wallet'},
 {'prompt': '[USER]: How can I track my order?',
  'response': '[BOT]: You can track your order from app. If you have trackind id you can track your order by entering trackid id on Webpage of respective courier service.'},
 {'prompt': '[USER]: Can I cancel or modify my order?',
  'response': '[BOT]: Yes! you can cancel or modify your order.'},
 {'prompt': '[USER]: What is your return or exchange policy?',
  'response': '[BOT]: The amount is refunded in th

In [39]:
conversation

[['[USER]: How do I place an order on your website?',
  '[BOT]: Shopping is super easy now. Register your Account today and start shopping by following these steps. 1. select item. 2. click buy now/ add to cart. 3. proceed to payment.4. select payment method'],
 ['[USER]: What are the payment options available?',
  '[BOT]: Goods can be ordered via pre-payment methods and via COD. Below are the payment methods available for Digital Goods:Credit/Debit Card ,Bank Direct Transfer and e-Wallet'],
 ['[USER]: How can I track my order?',
  '[BOT]: You can track your order from app. If you have trackind id you can track your order by entering trackid id on Webpage of respective courier service.'],
 ['[USER]: Can I cancel or modify my order?',
  '[BOT]: Yes! you can cancel or modify your order.'],
 ['[USER]: What is your return or exchange policy?',
  '[BOT]: The amount is refunded in the following scenarios: Refund against Return orders: the refund is processed if your return claim is deemed va

In [32]:
len(pre_prompt)

18730

# GREEDY RESPONSE

In [90]:
prompt='''
[USER]: How do I place an order on your website?
[BOT]: Shopping is super easy now. Register your Account today and start shopping by following these steps. 1. select item. 2. click buy now/ add to cart. 3. proceed to payment.4. select payment method[END]
[USER]: What are the payment options available?
[BOT]: Goods can be ordered via pre-payment methods and via COD. Below are the payment methods available for Digital Goods:Credit/Debit Card ,Bank Direct Transfer and e-Wallet[END]
[USER]: How can I track my order?
[BOT]: You can track your order from app. If you have trackind id you can track your order by entering trackid id on Webpage of respective courier service.[END]
[USER]: Can I cancel or modify my order?
[BOT]: Yes! you can cancel or modify your order.[END]
[USER]: What is your return or exchange policy?
[BOT]: The amount is refunded in the following scenarios: Refund against Return orders: the refund is processed if your return claim is deemed valid. Refund against Cancelled orders: Refund is automatically triggered once the cancellation has been successfully processed.[END]
[USER]: How long does it take for my order to be delivered?
[BOT]: You will receive your digital goods on the registered email address within 24hrs. Physical items mostly take 2-3 days for delivery.[END]
[USER]: What should I do if my order is damaged or defective?
[BOT]: Provide us with your order number and any relevant details, such as photos or videos of the issue, so that we can investigate and determine the best course of action. We may offer a refund, replacement item, or repair service depending on the circumstances.[END]
[USER]: Can I get a refund if I'm not satisfied with my purchase? 
[BOT]: Yes! you can ger refund. We do offer refunds to customers who are not satisfied with their purchase.We want our customers to be completely happy with their orders and will work with them to make thingsright.[END]
[USER]: How can I contact your customer service team?
[BOT]: You can call us at XXX-XXX-XXXXXX anytime ( Mon - Sun ) or to connect to our Seller Support teamon Chat between 10:00 AM - 4:00 PM ( Mon - Sun). You can also mail us at support.abc@xyz.com.[END]
[USER]: Do you offer any discounts or promotions?
[BOT]: You can avail discounts by using vouchers. You can collect vouchers by making purchases andperforming tasks.[END]
[USER]: Is my personal and payment information secure on your website?
[BOT]: we take the security of our customers' personal and payment information very seriously. We use a variety of security measures to protect our website and the information you provide to us.We use SSL encryption to protect your personal and payment information during transmission, and we do not store your payment information on our servers. We use a third-party payment processor that is PCI-compliant to handle all payment transactions, which means your payment information is processed securely and in compliance with industry standards.[END]
[USER]: What are the shipping costs?
[BOT]: Shipping costs vary depending on factors such as the weight and size of the item, the shipping method selected, and the destination.  I suggest checking the product page or contacting customer support for more information on the shipping costs for your order.[END]
[USER]: How do I know if my order has been shipped?
[BOT]: you can typically check the status of your order by logging into your account on the website or by checking the confirmation email you received when you placed your order. The seller may also send you a tracking number once your order has been shipped, which you can use to track the status of your shipment online. If you have any questions or concerns about the status of your order, you can contact customer support for assistance.[END]
[USER]: Do you offer international shipping?
[BOT]: we may offer international shipping to certain countries. To find out if we offer international shipping to your location, you can check the shipping information on our website or during the checkout process. If your country is not listed as an available shipping destination, we may not be able to fulfill your order at this time. However, we are constantly working to expand our shipping capabilities and may be able to offer international shipping to your location in the future. Please feel free to contact our customer support team for more information about our shipping policies and options.[END]
[USER]: How do I leave feedback or a review about my purchase?
[BOT]: we value our customers' feedback and opinions. To leave feedback or a review about your purchase, you can navigate to the product page on our website and look for the "Feedback" section. From there, you can leave a rating, write a review, and share your experience with other customers. Your feedback helps us improve our products and services.[END]
[USER]: Can I place an order over the phone?
[BOT]: we do not offer phone orders at this time. However, you can easily place an order on our website using your computer or mobile device. Our website is designed to be user-friendly and easy to navigate, making it simple for you to find the products you need and complete your purchase.[END]
[USER]: How do I use a coupon or promo code?
[BOT]: During the checkout process, you will have the opportunity to enter your coupon code in a designated field. The field  is labeled as "Coupon Code". After you have entered the code, click "Apply" or "Redeem" to apply the discount to your order. The discounted price should then be reflected in your order total.If you still have any query, please contact customer support.[END]
[USER]: Do you have a loyalty program or rewards system?
[BOT]: You can earn points for every purchase you do. These points can be used for discounts in your future purchases.[END]
[USER]: Can I get assistance with product selection?
[BOT]: If you need assistance in choosing a product, our website have product descriptions, reviews, and other helpful information to guide your decision. Additionally, you can contact our customer support team for personalized assistance.[END]
[USER]: Do you offer gift wrapping or gift cards?
[BOT]: we may offer gift wrapping and/or gift cards for purchase. Gift wrapping and gift cards can make great additions to your purchase, especially if you are buying a gift for someone else. You can check our website or contact our customer support team to find out if we offer these services and how you can purchase them.[END]
[USER]: Are there any restrictions on shipping to my location?
[BOT]: there may be restrictions on shipping to certain locations due to legal, logistical, or other reasons. We recommend that you check our website or contact our customer support team to find out if we are able to ship to your location and if any restrictions apply. Some common reasons for shipping restrictions may include customs regulations, import restrictions, and hazardous materials restrictions. If your location is restricted, we apologize for any inconvenience and encourage you to explore alternative shipping options or products that are available in your region. Our priority is to ensure that we comply with all applicable laws and regulations and provide a positive shopping experience for all of our customers.[END]
[USER]: How do I initiate a warranty claim for a product?
[BOT]: if you need to initiate a warranty claim for a product purchased on our website, we recommend that you contact the manufacturer or the brand directly, as they are typically responsible for handling warranty claims. You can usually find contact information for the manufacturer or the brand on the product packaging, the instruction manual, or their website. When contacting the manufacturer or the brand, be sure to provide them with your proof of purchase, such as your order confirmation or receipt, and a description of the issue you are experiencing with the product. The manufacturer or the brand may require additional information or documentation to process your warranty claim, and they may provide you with instructions on how to return the product for repair or replacement.[END]
'''

In [2]:
log = ''
past_kv = None
next_id = None

while True:
    user_input = input(" ")
    if user_input.lower() in ["exit", "quit", "stop"]:
        break
    break_word = '[END]'
        
    response,past_kv,next_id = generate_response_greedy(user_input, short_pre_prompt + log,
                                        break_word,max_length=30000, name='[BOT]',
                                        past_key_vals=past_kv, next_id=next_id)


 how to order from your website


NameError: name 'generate_response_greedy' is not defined

# FINE TUNING

In [2]:
'''conversation = [
    ("How do I place an order on your website?", "Shopping is super easy now. Register your Account today and start shopping by following these steps\nselect item\nclick buy now/ add to cart\nproceed to payment\nselect payment method"),
    ("What are the payment options available?", "Goods can be ordered via pre-payment methods and via COD. Below are the payment methods available for Digital Goods:Credit/Debit Card ,Bank Direct Transfer and e-Wallet"),
    ("How can I track my order?", "You can track your order from app. If you have trackind id you can track your order by entering trackid id on Webpage of respective courier service."),
    ("Can I cancel or modify my order?", "Yes! you can cancel or modify your order.")
]'''


In [78]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import DataLoader, Dataset
class ConversationDataset(Dataset):
    def __init__(self, conversation, tokenizer):
        self.examples = []
        self.max_length = 512
        pad = [0]
        for i in range(len(conversation)):
            input_text = conversation[i][0]
            output_text = conversation[i][1]
            input_ids = tokenizer.encode(input_text, add_special_tokens=True, max_length=self.max_length, truncation=True)
            output_ids = tokenizer.encode(output_text, add_special_tokens=True, max_length=self.max_length, truncation=True)
            padding_length = self.max_length - len(input_ids)
            #input_ids += [tokenizer.pad_token_id] * padding_length
            input_ids += pad * padding_length
            padding_length = self.max_length - len(output_ids)
            #output_ids += [tokenizer.pad_token_id] * padding_length
            output_ids += pad * padding_length
            self.examples.append({
                'input_ids': torch.tensor(input_ids, dtype=torch.long),
                'attention_mask': torch.tensor([1] * len(input_ids), dtype=torch.long),
                'decoder_input_ids': torch.tensor(output_ids[:-1], dtype=torch.long),
                'decoder_attention_mask': torch.tensor([1] * len(output_ids[:-1]), dtype=torch.long),
                'labels': torch.tensor(output_ids, dtype=torch.long)
                #'labels': torch.tensor(output_ids[1:], dtype=torch.long)
            })
    
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, index):
        return self.examples[index]
    

# initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-410m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-410m")

# define the dataset and data loader
dataset = ConversationDataset(conversation, tokenizer)

print(dataset)

data_loader = DataLoader(dataset, batch_size=2, shuffle=True)

# define the optimizer and learning rate scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

# train the model
model.train()
for epoch in range(15):
    total_loss = 0
    for batch in data_loader:
        print(len(batch))
        optimizer.zero_grad()
        inputs = {
            'input_ids': batch['input_ids'],
            'attention_mask': batch['attention_mask'],
            #'decoder_input_ids': batch['decoder_input_ids'],
            #'decoder_attention_mask': batch['decoder_attention_mask'],
            'labels': batch['labels']
        }
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    scheduler.step()
    avg_loss = total_loss / len(dataset)
    print(f"Epoch {epoch + 1} loss: {avg_loss:.4f}")



5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 1 loss: 0.9736
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 2 loss: 0.4942
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 3 loss: 0.4697
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 4 loss: 0.4295
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 5 loss: 0.4046
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 6 loss: 0.3862
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 7 loss: 0.3893
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 8 loss: 0.3797
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 9 loss: 0.3724
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 10 loss: 0.3530
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 11 loss: 0.3392
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 12 loss: 0.3286
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 13 loss: 0.3262
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 14 loss: 0.3297
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
Epoch 15 lo

## dumping model into pickel file

In [79]:
# import pickle
# with open('CustomerServiceMODEL.pkl', 'wb') as f:
#     pickle.dump(model, f)

In [80]:
# prompt and generate response
prompt = "How can I order from your website"
input_ids = tokenizer.encode(prompt, return_tensors='pt')
outputs = model.generate(input_ids=input_ids, max_length=512, do_sample=True)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


How can I order from your website to order customers the ordering to the ordering ordering to to to to website to and to toorder on ordering ordering to the on to, to to have order will leave order to ordered ordering to theorder to and youorder, on order the section to this the to to this customers this and on our will of have customers toorder orders ordering to to to un to listed on to our to customers to of order section the wish and, order of for the To customers we to you orders Order order to orders by and customers to customers, ordered ordering to. customers of customers our and to orders ordered to to ordered ordering of To or of and you after our we listed to normal our on our on of to ordered customers ordered out on customers to be order orders to To for ordered our the we to Order ordering on and our in on to un and we to to To or on if our of thisorder this the and and customers, will we listed do, toorder is a to to of theorder will our we to order we and leave outside 

In [ ]:
torch.tensor(input_ids, dtype=torch.long)

In [ ]:
torch.tensor(None, dtype=torch.long)

In [42]:
l=[1,2,3]

l += [0]*3

In [43]:
l

[1, 2, 3, 0, 0, 0]